In [47]:
import torch
import torch.nn as nn
from nltk.tokenize import word_tokenize
from os import path
from math import sqrt

In [48]:
class Vocabulary:
	def __init__(self):
		self.vocabulary = set('')
		self.stoi = {'<N>':0, '<S>': 1, '<E>': 2}
		self.itos = {0:'<N>', 1:'<S>', 2: '<E>'}

	def add(self, v):
		if type(v) == str:
			self.vocabulary.add(v)
		elif type(v) == list:
			self.vocabulary = self.vocabulary.union(set(v)) 

	def create_mappings(self):
		self.stoi = {v:i+1 for i, v in enumerate(self.vocabulary)}
		self.itos = {i+1:v for i, v in enumerate(self.vocabulary)}

	def encode(self, s): 
		return [self.stoi[c] for c in s]
	
	def decode(self, i): 
		return [self.itos[n] for n in i]
	

class PreProcessor:
	def __init__(self):
		self.english_vocabulary = Vocabulary()
		self.cherokee_vocabulary = Vocabulary()
		self.cherokee = []
		self.english = []
		self.max_length = 0
		self.count = 0

	def load_text(self, file_name):
		data, language = [], file_name.split('.')[1]

		with open(file_name) as f:
			for line in f.readlines():
				sentence = ['<S>'] + word_tokenize(line) + ['<E>']

				if language == 'en':
					self.english_vocabulary.add(sentence)
				else:
					self.cherokee_vocabulary.add(sentence)

				self.max_length = max(self.max_length, len(sentence))
				data.append(sentence)
				self.count += 1 
		return data
	
	def get_data(self, file_set):
		cherokee = self.load_text(path.join('chr_en_data', f'{file_set}.chr'))
		english  = self.load_text(path.join('chr_en_data', f'{file_set}.en' ))
		assert len(cherokee) == len(english)
		self.cherokee += cherokee
		self.english  += english

		return cherokee, english

	
	def create_tensors(self):
		self.english_vocabulary.create_mappings()
		self.cherokee_vocabulary.create_mappings()

		english  = torch.zeros(size=(self.count//2, self.max_length), dtype=int)
		cherokee = torch.zeros(size=(self.count//2, self.max_length), dtype=int)

		for i, sen in enumerate(self.english):
			for j, v in enumerate(self.english_vocabulary.encode(sen)):
				english[i, j] = v
		
		for i, sen in enumerate(self.cherokee):
			for j, v in enumerate(self.cherokee_vocabulary.encode(sen)):
				cherokee[i, j] = v

		self.cherokee, self.english = cherokee, english


preprocessor = PreProcessor()
preprocessor.get_data('dev')
preprocessor.get_data('test')
preprocessor.get_data('train')
preprocessor.create_tensors()

test = word_tokenize('ᏣᏌᏙᏰᏃ ᎢᎦᎦᏛ ᏓᏳᏂᎷᏤᎵ ᏂᎬᎾᏛ ᏗᏁᎯ.')

assert preprocessor.cherokee_vocabulary.decode(preprocessor.cherokee_vocabulary.encode(test)) == test
assert preprocessor.english.shape == preprocessor.cherokee.shape

In [49]:
cherokee_vocab_size, english_vocab_size = len(preprocessor.cherokee_vocabulary.vocabulary), len(preprocessor.english_vocabulary.vocabulary)

print(f'Cherokee Vocabulary Size: {cherokee_vocab_size}')
print(f'English  Vocabulary Size: {english_vocab_size}')

Cherokee Vocabulary Size: 44566
English  Vocabulary Size: 13930


In [50]:
EMBEDDING_DIMENSIONS = 5
QKV_DIMENSIONS       = 5
SEQUENCE_LENGTH      = 5
ATTENTION_HEADS      = 2
DECODERS             = 2
ENCODERS             = 2

In [51]:
def mask_tensor(t):
	mask = torch.tril(torch.ones(size=(t.shape)))
	mask[mask==0], mask[mask==1] = float('-inf'), 0
	
	return t + mask

test = torch.randn(SEQUENCE_LENGTH, SEQUENCE_LENGTH)
print(test)
print(mask_tensor(test))

tensor([[-1.1818,  0.1564,  1.3544, -0.8017, -0.3809],
        [-1.2670,  0.4257,  0.7800,  0.0700,  3.7385],
        [-0.6393, -1.1789, -0.8509,  0.5497,  0.6079],
        [ 0.8433, -1.2656,  1.2605, -0.9949, -0.2771],
        [ 0.2440,  0.8973, -0.0450, -0.0186, -0.1758]])
tensor([[-1.1818,    -inf,    -inf,    -inf,    -inf],
        [-1.2670,  0.4257,    -inf,    -inf,    -inf],
        [-0.6393, -1.1789, -0.8509,    -inf,    -inf],
        [ 0.8433, -1.2656,  1.2605, -0.9949,    -inf],
        [ 0.2440,  0.8973, -0.0450, -0.0186, -0.1758]])


In [52]:
class AttentionHead(nn.Module):
	def __init__(self, masked=False):
		super().__init__()
		self.obtain_key   = nn.Linear(EMBEDDING_DIMENSIONS, QKV_DIMENSIONS)
		self.obtain_query = nn.Linear(EMBEDDING_DIMENSIONS, QKV_DIMENSIONS)
		self.obtain_value = nn.Linear(EMBEDDING_DIMENSIONS, QKV_DIMENSIONS)
		self.masked = masked

	def forward(self, data, encoder_output=None):
		if encoder_output is None: Q, K, V = self.obtain_query(data), self.obtain_key(data), self.obtain_value(data)
		else: Q, K, V = self.obtain_query(data), self.obtain_key(encoder_output), self.obtain_value(encoder_output)
		mat_mul = Q @ K.transpose(-2, -1)
		scaled_mat_mul = mat_mul / sqrt(QKV_DIMENSIONS)
		if self.masked: scaled_mat_mul = mask_tensor(scaled_mat_mul)
		softmax_mat_mul = torch.softmax(scaled_mat_mul, dim=-1)
		output = softmax_mat_mul @ V

		return output


test, e_output = torch.randn(size=(SEQUENCE_LENGTH, EMBEDDING_DIMENSIONS)), torch.randn(size=(SEQUENCE_LENGTH, QKV_DIMENSIONS))
test_module = AttentionHead()
print(test_module(test, encoder_output=e_output))



tensor([[-0.2319,  0.6263, -0.6390,  0.4614,  0.3419],
        [-0.2491,  0.7028, -0.5957,  0.5357,  0.3715],
        [-0.3282,  0.6951, -0.6529,  0.4929,  0.3771],
        [-0.3488,  0.7689, -0.6183,  0.5641,  0.4104],
        [-0.4014,  0.8183, -0.6225,  0.5954,  0.4409]], grad_fn=<MmBackward0>)


In [53]:
class MultiHeadedAttention(nn.Module):
	def __init__(self, masked=False):
		super().__init__()
		self.heads  = [AttentionHead(masked=masked) for _ in range(ATTENTION_HEADS)]
		self.linear = nn.Linear(EMBEDDING_DIMENSIONS*ATTENTION_HEADS, EMBEDDING_DIMENSIONS)

	def forward(self, data, encoder_output=None):
		vectors = torch.cat([head(data, encoder_output=encoder_output) for head in self.heads], dim=-1)
		return self.linear(vectors)
	

test = torch.randn(size=(SEQUENCE_LENGTH, EMBEDDING_DIMENSIONS))
test_module = MultiHeadedAttention()
print(test_module(test))


tensor([[ 0.0480, -0.2671,  0.0732,  0.0511,  0.3381],
        [ 0.0240, -0.2793,  0.0619,  0.0437,  0.3097],
        [ 0.0531, -0.2084,  0.0747,  0.0535,  0.3646],
        [ 0.0841, -0.2314,  0.0860,  0.0677,  0.3854],
        [ 0.0465, -0.2300,  0.0710,  0.0444,  0.3553]],
       grad_fn=<AddmmBackward0>)


In [54]:
class FeedForward(nn.Module):
	def __init__(self):
		super().__init__()
		self.network = nn.Sequential(
			nn.Linear(EMBEDDING_DIMENSIONS, EMBEDDING_DIMENSIONS),
			nn.ReLU(),
			nn.Linear(EMBEDDING_DIMENSIONS, EMBEDDING_DIMENSIONS)
		)

	def forward(self, data):
		return self.network(data)

test = torch.randn(size=(SEQUENCE_LENGTH, EMBEDDING_DIMENSIONS))
test_module = FeedForward()
print(test_module(test))


tensor([[ 0.0742, -0.3341,  0.0483,  0.0585,  0.3479],
        [ 0.2205, -0.0472, -0.0607,  0.4060,  0.3039],
        [ 0.1778, -0.0379, -0.1388,  0.4281,  0.1861],
        [-0.1374, -0.5493,  0.1107,  0.0517,  0.0453],
        [-0.1366, -0.6718,  0.2085, -0.1662,  0.3717]],
       grad_fn=<AddmmBackward0>)


In [71]:
class Encoder(nn.Module):
	def __init__(self):
		super().__init__()
		self.multi_headed_attention = MultiHeadedAttention()
		self.norm1 = nn.LayerNorm(EMBEDDING_DIMENSIONS)
		self.feed_forward = FeedForward()
		self.norm2 = nn.LayerNorm(EMBEDDING_DIMENSIONS)

	def forward(self, data):
		attention_vectors = self.multi_headed_attention(data)
		normalised = self.norm1(attention_vectors) + data # Residual Connection
		fed_through = self.feed_forward(normalised)
		normalised_2 = self.norm2(fed_through) + normalised

		return normalised_2
	
test = torch.randn(size=(SEQUENCE_LENGTH, EMBEDDING_DIMENSIONS))
print(test)
test_module = Encoder()
print(test_module(test))

tensor([[-1.4355,  1.6871, -0.4420,  0.0237, -0.5343],
        [ 0.9636, -0.8538,  1.4486,  1.2565,  0.6619],
        [-0.3767,  0.6192, -0.6057,  0.1669, -0.2945],
        [ 1.9749,  0.4844,  0.0273,  0.9701, -0.0839],
        [-1.2214,  2.1595,  0.4665,  0.9157,  0.0314]])
tensor([[ 0.8310, -0.8541, -0.7902, -0.5329,  0.6453],
        [ 3.6102, -3.7986,  1.3924,  1.1501,  1.1227],
        [ 2.0784, -1.9762, -1.0875, -0.2214,  0.7157],
        [ 4.5657, -2.5769, -0.0849,  1.1932,  0.2757],
        [ 1.2391, -0.5895,  0.6425,  0.3780,  0.6814]], grad_fn=<AddBackward0>)


In [101]:
class Decoder(nn.Module):
	def __init__(self):
		super().__init__()
		self.masked_attention = MultiHeadedAttention(masked=True)
		self.norm1 = nn.LayerNorm(EMBEDDING_DIMENSIONS)
		self.cross_attention  = MultiHeadedAttention()
		self.norm2 = nn.LayerNorm(EMBEDDING_DIMENSIONS)
		self.feed_forward = FeedForward()
		self.norm3 = nn.LayerNorm(EMBEDDING_DIMENSIONS)

	def forward(self, data, encoder_output):
		attention_vectors = self.masked_attention(data)
		normalised = self.norm1(attention_vectors) + data
		cross_attention = self.cross_attention(normalised, encoder_output=encoder_output)
		normalised = self.norm2(cross_attention) + normalised
		linear = self.feed_forward(normalised)
		normalised = self.norm3(linear) + normalised

		return normalised


test = torch.randn(size=(EMBEDDING_DIMENSIONS, SEQUENCE_LENGTH))
encoder_output = torch.randn(size=(SEQUENCE_LENGTH, QKV_DIMENSIONS))
test_module = Decoder()
print(test_module(test, encoder_output))

tensor([[-3.8578, -0.6436,  3.1509,  1.9776, -4.1137],
        [-1.3637, -0.1205,  4.1757,  1.7826, -2.6260],
        [-2.0298, -0.4894,  3.5091,  3.8996, -5.2450],
        [-1.1757, -1.2245,  1.2899,  1.5133, -2.1386],
        [-0.0238,  0.0182,  2.1690,  2.6096, -0.7285]], grad_fn=<AddBackward0>)
